In [1]:
import os
import pandas as pd
import re


### Find all modules

In [2]:
src_folder = os.getcwd()  
modules = [f.name for f in os.scandir(src_folder) if f.is_dir() and not f.name.startswith('.')]

In [3]:
len(modules)

190

In [4]:
data = []
for module in modules:
    sub_modules = [f.name for f in os.scandir(module) if f.is_dir() and f.name.endswith('.Autorest')]
    print(f"module: {module} - sub_module: {",".join(sub_modules)}")
    if sub_modules:
        for sub_module in sub_modules:
            data.append((module,sub_module))
    else:
        data.append((module, None))

module: Accounts - sub_module: 
module: ADDomainServices - sub_module: ADDomainServices.Autorest
module: Advisor - sub_module: Advisor.Autorest
module: Aks - sub_module: Aks.Autorest
module: AksArc - sub_module: AksArc.Autorest
module: Alb - sub_module: Alb.Autorest
module: AlertsManagement - sub_module: PrometheusRuleGroups.Autorest
module: AnalysisServices - sub_module: 
module: ApiManagement - sub_module: 
module: App - sub_module: App.Autorest
module: AppComplianceAutomation - sub_module: AppComplianceAutomation.Autorest
module: AppConfiguration - sub_module: AppConfiguration.Autorest,AppConfigurationData.Autorest
module: ApplicationInsights - sub_module: ApplicationInsights.Autorest
module: ArcGateway - sub_module: ArcGateway.Autorest
module: ArcResourceBridge - sub_module: ArcResourceBridge.Autorest
module: Astro - sub_module: Astro.Autorest
module: Attestation - sub_module: Attestation.Autorest
module: Automanage - sub_module: Automanage.Autorest
module: Automation - sub_module:

In [5]:
df = pd.DataFrame(data, columns=["module", "sub_module"])
df.head()

,module,sub_module
0,Accounts,None
1,ADDomainServices,ADDomainServices.Autorest
2,Advisor,Advisor.Autorest
3,Aks,Aks.Autorest
4,AksArc,AksArc.Autorest


In [6]:
df["v_autorest"] = "v4"
df["GA"] = False
df["Status"] = None
df.head()

,module,sub_module,v_autorest,GA,Status
0,Accounts,None,v4,False,None
1,ADDomainServices,ADDomainServices.Autorest,v4,False,None
2,Advisor,Advisor.Autorest,v4,False,None
3,Aks,Aks.Autorest,v4,False,None
4,AksArc,AksArc.Autorest,v4,False,None


### Find autorest version

In [7]:
for index, row in df.iterrows():
    if row["sub_module"] == None:
        df.at[index, "v_autorest"] = None
        continue
        
    current_dir = os.getcwd() 
    readme_path = os.path.join(current_dir, row["module"], row["sub_module"], "README.md")

    if not os.path.exists(readme_path):
        raise FileNotFoundError(f"The readme file for module: {row["module"]} sub_module： {row["sub_module"]} does not exist.")

    with open(readme_path, 'r', encoding='utf-8') as file:
        content = file.read()

        pattern = r'use-extension:\s*"\@autorest/powershell":\s*"3.x"'
        match = re.search(pattern, content, re.MULTILINE)

        if match:
            # print(f"V3 tag found in module: {row["module"]} sub_module： {row["sub_module"]}")
            df.at[index, "v_autorest"] = "v3"

In [8]:
df.head()

,module,sub_module,v_autorest,GA,Status
0,Accounts,None,None,False,None
1,ADDomainServices,ADDomainServices.Autorest,v3,False,None
2,Advisor,Advisor.Autorest,v3,False,None
3,Aks,Aks.Autorest,v3,False,None
4,AksArc,AksArc.Autorest,v4,False,None


In [9]:
df.loc[df['v_autorest'] == 'v3'].shape

(106, 5)

### Find GA modules

In [10]:
pattern = r"ModuleName\s*=\s*'([^']+)';\s*RequiredVersion\s*=\s*'([^']+)'"

def extract_modules_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            match = re.search(pattern, line)
            if match:
                module_name = match.group(1).replace('Az.', '')
                required_version = match.group(2)
                print(f"ModuleName: {module_name}, RequiredVersion: {required_version}")
                
                df.loc[df['module'] == module_name, 'GA'] = True

current_dir = os.getcwd()
file_path = os.path.join(current_dir, "..", "tools/Az/Az.psd1")

extract_modules_from_file(file_path)

ModuleName: Advisor, RequiredVersion: 2.1.0
ModuleName: Aks, RequiredVersion: 6.1.0
ModuleName: AnalysisServices, RequiredVersion: 1.2.0
ModuleName: ApiManagement, RequiredVersion: 4.1.0
ModuleName: App, RequiredVersion: 2.0.1
ModuleName: AppConfiguration, RequiredVersion: 1.4.0
ModuleName: ApplicationInsights, RequiredVersion: 2.3.0
ModuleName: ArcResourceBridge, RequiredVersion: 1.1.0
ModuleName: Attestation, RequiredVersion: 2.1.0
ModuleName: Automanage, RequiredVersion: 1.1.0
ModuleName: Automation, RequiredVersion: 1.11.1
ModuleName: Batch, RequiredVersion: 3.7.0
ModuleName: Billing, RequiredVersion: 2.2.0
ModuleName: Cdn, RequiredVersion: 3.3.0
ModuleName: CloudService, RequiredVersion: 2.1.0
ModuleName: CognitiveServices, RequiredVersion: 1.15.0
ModuleName: Compute, RequiredVersion: 9.1.0
ModuleName: ConfidentialLedger, RequiredVersion: 1.1.0
ModuleName: ConnectedMachine, RequiredVersion: 1.1.1
ModuleName: ContainerInstance, RequiredVersion: 4.1.1
ModuleName: ContainerRegistry, 

In [11]:
df.head()

,module,sub_module,v_autorest,GA,Status
0,Accounts,None,None,False,None
1,ADDomainServices,ADDomainServices.Autorest,v3,False,None
2,Advisor,Advisor.Autorest,v3,True,None
3,Aks,Aks.Autorest,v3,True,None
4,AksArc,AksArc.Autorest,v4,False,None


In [12]:
n_modules = df['module'].nunique()
n_v3_sub_modules = df.loc[df['v_autorest'] == 'v3'].shape[0]
n_v3_modules = df.loc[df['v_autorest'] == 'v3']['module'].nunique()

print(f"Total number of modules are {n_modules}.")
print(f"Among them, total number of v3 modules are {n_v3_modules}.")
print(f"Within v3 modules, there are {n_v3_sub_modules} actual sub modules need to be upgraded.")

Total number of modules are 190.
Among them, total number of v3 modules are 100.
Within v3 modules, there are 106 actual sub modules need to be upgraded.


In [13]:
ga_modules = df.loc[df['GA'] == True]['module'].nunique()
v3_ga_modules = df.loc[(df['GA'] == True) & (df['v_autorest'] == 'v3')]['module'].nunique()
v3_ga_sub_modules = df.loc[(df['GA'] == True) & (df['v_autorest'] == 'v3')].shape[0]

print(f"There are total {v3_ga_modules} V3 GA modules, can be further devided into {v3_ga_sub_modules} V3 GA sub modules that should be upgraded first.")

There are total 45 V3 GA modules, can be further devided into 50 V3 GA sub modules that should be upgraded first.


In [17]:
n_v4_modules = df.loc[df['v_autorest'] == 'v4']['module'].nunique()
n_v4_sub_modules = df.loc[df['v_autorest'] == 'v4'].shape[0]
print(f"There are total {n_v4_modules} V4 modules, can be further devided into {n_v4_sub_modules} V4 sub modules.")

There are total 47 V4 modules, can be further devided into 54 V4 sub modules.


In [19]:
v3_nga_sub_modules = df.loc[(df['GA'] == False) & (df['v_autorest'] == 'v3')].shape[0]
print(f"There are total {v3_nga_sub_modules} v3 pre-GA modules")

There are total 56 v3 pre-GA modules


In [14]:
df.to_csv('modules_audit.csv', index=False)

### Sort on autorest version and GA 

In [15]:
v_autorest_order = {'v3': 0, 'v4': 1, 'none': 2}
df['v_autorest_sort'] = df['v_autorest'].map(v_autorest_order)
df_sorted = df.sort_values(by=['v_autorest_sort', 'GA'], ascending=[True, False])
df_sorted = df_sorted.drop(columns=['v_autorest_sort'])
df_sorted.to_csv('modules_audit_sorted.csv', index=False)